In [1]:
%matplotlib qt
import matplotlib.pyplot as plt
import os
import mne
from mne_bids import make_bids_basename
from IPython.display import Image
from mayavi import mlab

from nilearn import plotting

mne.set_log_level('WARNING')

In [2]:
sSub, session, task, run = '08', '01', 'picturenaming', '01'
bids_basename = make_bids_basename(subject=sSub, session=session,
                                  task=task, run=run)
data_dir = '/Users/chholak/data/pic-name-data-bids/'
data_path = data_dir + 'MEG/'
bids_path = data_path + 'sub-' + sSub + '/ses-' + session + '/meg/'

raw_fname = bids_path + bids_basename + '_meg.fif'

subjects_dir = os.path.join(data_dir, 'MRI/')
subject = 'sub-' + sSub

In [3]:
t1_fname = os.path.join(data_dir, 'MRI/sub-' + sSub + '/mri/T1.mgz')

In [ ]:
plotting.plot_anat(t1_fname);
plt.show()

In [ ]:
mne.viz.plot_bem(subject=subject, subjects_dir=subjects_dir,
                orientation='coronal')

In [ ]:
# mne.gui.coregistration(subject=subject, subjects_dir=subjects_dir, inst=raw_fname);

In [4]:
trans_fname = os.path.join(bids_path, subject + '-trans.fif')
info = mne.io.read_info(raw_fname)
fig = mne.viz.plot_alignment(info, trans_fname, subject=subject, dig=True,
                            subjects_dir=subjects_dir, verbose=True)

Using outer_skin.surf for head surface.
Getting helmet for system 306m
Using mayavi 3d backend.



In [ ]:
mlab.close('all')

In [5]:
src = mne.setup_source_space(subject, spacing='oct6', subjects_dir=subjects_dir,
                            add_dist=False)

In [7]:
# fig = mne.viz.plot_alignment(info, trans_fname, subject=subject, src=src,
#                             subjects_dir=subjects_dir, verbose=True, meg=False,
#                             eeg=False);

Using outer_skin.surf for head surface.


In [8]:
mne.viz.plot_alignment(info, trans_fname, subject=subject,
                      src=src, subjects_dir=subjects_dir, dig=True,
                      surfaces=['head', 'white'], coord_frame='meg', meg=False)

In [ ]:
mlab.close()

In [9]:
conductivity = (0.3,)
model = mne.make_bem_model(subject=subject, ico=4, conductivity=conductivity,
                          subjects_dir=subjects_dir)
bem = mne.make_bem_solution(model)

In [11]:
bem_fname = os.path.join(bids_path, subject + '-bem.fif')
mne.bem.write_bem_solution(bem_fname, bem)

In [12]:
fwd = mne.make_forward_solution(raw_fname, trans=trans_fname, src=src,
                               bem=bem, meg=True, eeg=False, mindist=5.0, n_jobs=1)

In [13]:
fwd_fname = os.path.join(bids_path, subject + '-meg-fwd.fif')
mne.write_forward_solution(fwd_fname, fwd, overwrite=True)

In [14]:
fwd = mne.read_forward_solution(fwd_fname)
fwd

<Forward | MEG channels: 306 | EEG channels: 0 | Source space: Surface with 8196 vertices | Source orientation: Free>

In [15]:
fwd = mne.convert_forward_solution(fwd, surf_ori=True)
leadfield = fwd['sol']['data']
print("Leadfield size : %d sensors x %d dipoles" % leadfield.shape)

Leadfield size : 306 sensors x 24588 dipoles


In [16]:
# sens_map = mne.sensitivity_map(fwd, ch_type='grad', mode='fixed')

In [17]:
# picks = mne.pick_types(fwd['info'], meg='grad', eeg=False)
# im = plt.imshow(leadfield[picks, :500], origin='lower', aspect='auto', cmap='RdBu_r')
# plt.xlabel('sources')
# plt.ylabel('sensors')
# plt.title('Leadfield matrix for Gradiometers', fontsize=14)
# plt.colorbar(cmap='RdBu_r')

In [18]:
# plt.figure()
# plt.hist(sens_map.data.ravel(), bins=20, label='Gradiometers', color='c')
# plt.legend()
# plt.title('Normal orientation sensitivity')
# plt.xlabel('sensitivity')
# plt.ylabel('count');

In [19]:
# clim = dict(kind='percent', lims=(0, 50, 99), smoothing_steps=3)
# brain = sens_map.plot(subject=subject, time_label='GRAD sensitivity',
#                      subjects_dir=subjects_dir, clim=clim, smoothing_steps=8);
# view = 'lat'
# brain.show_view(view)

((180.0, 90.0, 432.1112365722656, array([0., 0., 0.])), 90.0)

In [ ]:
# brain.save_image('sensitivity_map_grad_%s.jpg' % view)
# Image(filename='sensitivity_map_grad_%s.jpg' % view, width=400)